In [45]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, calinski_harabasz_score, davies_bouldin_score, silhouette_score


In [46]:
repeat_customers_w_norm_df = pd.read_feather("repeat_customers_w_norm_df.ftr")

with pd.option_context("display.max_columns", None):
    display(repeat_customers_w_norm_df)

,customer_unique_id,total_orders,total_prods_ordered,first_customer_review_score,avg_customer_review_score,reordered,high_first_rating,order_ids,order_purchase_timestamps,total_spent,order_delay_days,years_since_last_purchase,months_since_last_purchase,orders_per_year,orders_per_month,spent_per_year,spent_per_month,proport_payments_by_credit_card,proport_payments_by_boleto,proport_payments_by_voucher,proport_payments_by_debit_card,proport_payments_by_not_defined,proport_orders_paid_in_install,proport_orders_paid_lump_sum,total_orders_norm,total_prods_ordered_norm,first_customer_review_score_norm,avg_customer_review_score_norm,total_spent_norm,order_delay_days_norm,months_since_last_purchase_norm,orders_per_month_norm,spent_per_month_norm,proport_orders_paid_in_install_norm,proport_orders_paid_lump_sum_norm,k_means_cluster
0,4c93744516667ad3b8f1fb645a3116a4,2,2,5.0,5.0,True,True,"[5741ea1f91b5fbab2bd2dc653a5b5099, 98b737f8bd0...","[2017-10-15T12:44:13.000000, 2017-09-14T18:14:...",565.88,30.770625,0.867168,10.406011,2.098210,0.174851,593.667462,49.472289,0.5,0.0,0.0,0.5,0.0,0.0,1.0,0.0,0.000000,1.00,1.000,0.070324,5.052823e-02,0.459602,0.008728,0.007226,0.0,1.0,1
1,1175e95fb47ddff9de6b2b06188f7e0d,2,2,5.0,5.0,True,True,"[7845a2492ab1b4f2cf3d56c7b8da1446, 1ebeea841c5...","[2018-01-18T12:35:45.000000, 2018-01-18T12:35:...",83.19,0.000012,0.607621,7.291449,3.284782,0.273732,136.630492,11.385874,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,1.00,1.000,0.006270,1.900571e-08,0.322041,0.018642,0.001436,0.0,1.0,1
2,295c05e81917928d76245e842748184d,2,2,3.0,4.0,True,False,"[cf5e115a19c54f0457aa4c00fb88ae63, 852d2f4d377...","[2018-06-06T11:59:53.000000, 2018-03-07T15:57:...",281.50,90.835174,0.227907,2.734889,4.192270,0.349356,590.061971,49.171831,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.50,0.750,0.032586,1.491598e-01,0.120792,0.026224,0.007180,1.0,0.0,0
3,e9dd12dca17352644a959d9dea133935,2,2,3.0,4.0,True,False,"[19c0c76962278b152cc44fc63ffd5e48, f916cacb752...","[2018-04-18T09:42:53.000000, 2018-03-05T18:12:...",113.83,43.646273,0.362047,4.344565,4.146904,0.345575,236.021030,19.668419,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.50,0.750,0.010336,7.167124e-02,0.191886,0.025845,0.002695,0.0,1.0,1
4,e079b18794454de9d2be5c12b4392294,2,6,5.0,5.0,True,True,"[77b062be7c5bd21712905feb8e1cfeed, 7e3fefb35b0...","[2017-06-14T18:52:29.000000, 2017-06-14T18:31:...",437.78,0.014294,1.202534,14.430412,1.659692,0.138308,363.290007,30.274167,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.181818,1.00,1.000,0.053325,2.347205e-05,0.637348,0.005064,0.004308,1.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2796,574e36a816bf7a295404e686062fc0de,2,3,4.0,4.5,True,True,"[25c0762610a416a21571e4ab3cf179b8, 325c8590cc2...","[2018-04-26T18:07:40.000000, 2017-06-06T17:00:...",222.86,324.046782,0.339231,4.070777,1.629833,0.135819,181.612268,15.134356,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.045455,0.75,0.875,0.024805,5.321149e-01,0.179794,0.004814,0.002006,1.0,0.0,2
2797,76f7d40948174f5cde0fcf8e6147b59d,2,4,2.0,2.0,True,False,"[09d49d145e25e27490765a4085a8e720, b95ee76467f...","[2018-04-29T22:59:41.000000, 2017-12-30T00:06:...",373.60,120.953472,0.330481,3.965768,3.019726,0.251644,564.084767,47.007064,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.090909,0.25,0.250,0.044808,1.986168e-01,0.175156,0.016427,0.006851,0.0,1.0,1
2798,88fcb680dc8cd2e540010c0750b78242,2,2,4.0,4.0,True,True,"[eda613a4e8989dac08362e3326431eaa, 00772b2af35...","[2018-01-19T08:02:06.000000, 2017-04-24T10:53:...",118.61,269.881181,0.605408,7.264893,1.486386,0.123866,88.150136,7.345845,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.75,0.750,0.010970,4.431700e-01,0.320868,0.003616,0.000822,1.0,0.0,2
2799,2dcbfb99d95c7ba6eb992a98627f45b8,2,3,1.0,1.0,True,False,"[ce5f998d724ed1ac61c890e33b24a33f, 4035b52134b...","[2017-12-03T20:37:09.000000, 2017-11-28T01:16:...",285.23,5.806042,0.732390,8.788685,2.667412,0.222284,380.412988,31.701082,

In [47]:
all_timestamps = sorted(
    list(
        np.concatenate(
            repeat_customers_w_norm_df["order_purchase_timestamps"].to_list()
        ).flat
    )
)
first_timestamp = all_timestamps[0]
timestamp_to_add = pd.Timestamp(first_timestamp) + pd.DateOffset(years=1)
timestamps_to_compare = [timestamp_to_add]

for i in range(1, 13):
    timestamp_to_add = pd.Timestamp(timestamp_to_add) + pd.DateOffset(months=1)
    timestamps_to_compare += [timestamp_to_add]

timestamps_to_compare = [x for x in timestamps_to_compare if x < all_timestamps[-1]] + [
    pd.Timestamp(all_timestamps[-1])
]
timestamp_to_comp_months = [12 + i for i in range(0, len(timestamps_to_compare))]
dfs_to_timestamps = []

for timestamp in timestamps_to_compare:
    df = repeat_customers_w_norm_df.copy()
    idxs_to_drop = []

    for index, row in df.iterrows():
        if len([x for x in row["order_purchase_timestamps"] if x <= timestamp]) < 2:
            idxs_to_drop += [index]

    dfs_to_timestamps += [df.drop(idxs_to_drop)]

num_of_entries = [df.shape[0] for df in dfs_to_timestamps]
normed_feats = [
    "total_orders_norm",
    "total_prods_ordered_norm",
    "first_customer_review_score_norm",
    "avg_customer_review_score_norm",
    "total_spent_norm",
    "order_delay_days_norm",
    "months_since_last_purchase_norm",
    "orders_per_month_norm",
    "spent_per_month_norm",
    "proport_orders_paid_in_install_norm",
    "proport_orders_paid_lump_sum_norm",
]

for df in dfs_to_timestamps:
    df["orig_k_means_cluster"] = df["k_means_cluster"]
    # df["k_means_cluster"] = KMeans(n_clusters=3, n_init=10, random_state=0).fit_predict(
    #     df[normed_feats]
    # )
    df["k_means_cluster"] = KMeans(n_clusters=3, n_init=10).fit_predict(
        df[normed_feats]
    )


In [48]:
silh_scores = []
ch_scores = []
db_scores = []
ari_scores = []

for df in dfs_to_timestamps:
    silh_scores += [
        silhouette_score(
            df[normed_feats].to_numpy(),
            df["k_means_cluster"].to_list(),
            metric="euclidean",
        )
    ]
    ch_scores += [
        calinski_harabasz_score(
            df[normed_feats].to_numpy(), df["k_means_cluster"].to_list()
        )
    ]
    db_scores += [
        davies_bouldin_score(
            df[normed_feats].to_numpy(), df["k_means_cluster"].to_list()
        )
    ]
    ari_scores += [
        adjusted_rand_score(
            df["orig_k_means_cluster"].to_list(), df["k_means_cluster"].to_list()
        )
    ]


cluster_metrics_df = pd.DataFrame(
    {
        "months_from_start": timestamp_to_comp_months,
        "max_timestamp": timestamps_to_compare,
        "number_of_entries": num_of_entries,
        "silhouette_scores": silh_scores,
        "calinski_harabasz_scores": ch_scores,
        "davies_bouldin_scores": db_scores,
        "adjusted_rand_scores": ari_scores,
    }
)

display(cluster_metrics_df)


,months_from_start,max_timestamp,number_of_entries,silhouette_scores,calinski_harabasz_scores,davies_bouldin_scores,adjusted_rand_scores
0,12,2017-10-03 09:44:50,688,0.492309,787.023153,0.834274,1.000000
1,13,2017-11-03 09:44:50,822,0.534147,901.212701,0.933649,0.489829
2,14,2017-12-03 09:44:50,1039,0.522218,1097.703519,0.935367,0.485737
3,15,2018-01-03 09:44:50,1188,0.512489,1209.806849,0.943467,0.457832
4,16,2018-02-03 09:44:50,1403,0.466842,1460.468420,0.868417,0.984820
5,17,2018-03-03 09:44:50,1645,0.462256,1691.513772,0.869912,0.980946
6,18,2018-04-03 09:44:50,1839,0.456101,1848.638155,0.881348,0.982869
7,19,2018-05-03 09:44:50,2030,0.448826,1974.772604,0.909663,0.984247
8,20,2018-06-03 09:44:50,2239,0.440146,2095.274101,0.936805,0.983544
9,21,2018-07-03 09:44:50,2438,0.433649,2209.597177,0.953168,0.982774
